In [1]:
import sys, os, decimal, json
sys.path.append(os.path.realpath('..'))
import json, time
import pandas as pd
import random
import matplotlib.pyplot as plt
import glob

import matplotlib.pyplot as plt
import numpy as np
import PIL, json
import tensorflow as tf
import socket
import pathlib
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageOps

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.applications import VGG16

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

from TrainingPipelines.ImageClassifier.ModelBuilder import build_and_train

import seaborn as sns
import mplcyberpunk
tf.random.set_seed(42)

plt.style.use("cyberpunk")
plt.rcParams['figure.facecolor'] = '#0d1117'
plt.rcParams['axes.facecolor'] = '#0d1117'
plt.rcParams['savefig.facecolor'] = '#0d1117'

# PLAN
1. Train Boxes
2. transfer in those boxes
3. train location of boxes

In [14]:
config_meta = {'image_resize': [20, 8],
     'class_location': 'O:\\eve_models\\training_data\\box_counter\\box_counter_classes.json',
     'model_location': 'O:\\eve_models\\training_data\\box_counter\\box_counter_model.h5',
     'class_names': ['1', '2'],
     'cm': [[ 70,   0],
            [  0, 115]]}
box_counter_model = tf.keras.models.load_model(config_meta['model_location'], compile=False)
config = json.load(open(config_meta['class_location']))
config

['1', '2']

In [5]:
box_counter_model.layers

In [21]:
img = Image.open(r"O:\eve_models\training_data\box_counter\2\0a560779-d4cf-11ed-bc5e-2cf05d9fe8eb.png")
train_x = []

img = img.resize(
    (config_meta['image_resize'][1], config_meta['image_resize'][0]),
    # TF trains backwards
    resample=Image.Resampling.NEAREST)

train_x.append(np.array(img))

image_array = train_x[0]

In [60]:
image_array.shape

(20, 8, 3)

In [17]:
prediction = box_counter_model.predict(np.array([image_array]))
prediction

1/1 [==============================] - 0s 16ms/step


array([[-11.821878,  15.709077]], dtype=float32)

In [20]:
img

In [19]:
config[np.argmax(prediction)]

'2'

In [28]:
conv_model = tf.keras.Model(box_counter_model.layers[0:-4])

In [31]:
layer_outputs = []
for i in range(1, len(box_counter_model.layers)):
    tmp_model = tf.keras.Model(box_counter_model.layers[0].input, box_counter_model.layers[i].output)
    tmp_output = tmp_model.predict(np.array([image_array]))[0]
    layer_outputs.append(tmp_output)

1/1 [==============================] - 0s 48ms/step


In [83]:
layer_outputs[3].shape

(5, 2, 32)

In [71]:
layer_outputs[3][:,:,0:3].shape

(5, 2, 3)

In [84]:
single = layer_outputs[0][:,:,0:3]
single = (single * np.array(1000)).astype(np.uint8)
single.shape

(20, 8, 3)

In [85]:
Image.fromarray(single).convert('RGB')

In [87]:
source_model = tf.keras.Model(box_counter_model.layers[0].input, box_counter_model.layers[7].output)
for source_layer in source_model.layers:
    source_model.trainable = False
tmp_output = source_model.predict(np.array([image_array]))[0]
tmp_output

1/1 [==============================] - 0s 42ms/step


array([0.23544717, 0.        , 0.97299355, 0.11945027, 0.        ,
       0.00321241, 0.        , 0.3977048 , 1.383597  , 0.        ,
       0.        , 0.        , 0.92766416, 0.        , 1.0278487 ,
       0.        , 0.        , 1.1910099 , 1.607712  , 0.        ,
       0.        , 0.        , 1.5728966 , 0.        , 1.0251459 ,
       0.41926244, 1.0394814 , 1.0289046 , 1.6481133 , 0.98409426,
       0.        , 0.        , 0.24609137, 0.7172504 , 0.86423695,
       0.        , 0.        , 1.6230892 , 0.        , 0.        ,
       0.        , 0.31238276, 0.48728997, 0.        , 1.1716206 ,
       0.        , 1.1728804 , 0.        , 0.        , 0.        ,
       0.        , 1.0072582 , 0.37619072, 0.30103597, 0.37740222,
       0.        , 1.1557908 , 0.        , 0.81128174, 0.        ,
       1.4141653 , 0.71562594, 1.2477237 , 0.70214367, 0.5549363 ,
       0.8446619 , 1.0294468 , 0.        , 0.33627737, 0.6627551 ,
       0.5108382 , 0.10177048, 0.3363555 , 0.72705054, 0.     

In [ ]:
model = Sequential([
    layers.Input(shape=(img_height, img_width,3)),
    #layers.Rescaling(1. / 255, input_shape=(img_height, img_width,3)),
    source_model,
    layers.Flatten(),
    #layers.Dense(units=128, activation='relu'),
    #layers.Dense(units=64, activation='relu'),
    layers.Dense(units=32, activation="relu"),
    layers.Dense(units=1, activation="sigmoid", name='bounding_box')  
])
